# Проект исследования поведения клиентов Бета-Банка

## Описание проекта.

Из банка заказчика стали уходить клиенты. 
Банковские маркетологи пришли к выводу что, сохранять текущих клиентов дешевле, чем привлекать новых.
Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет на основе предоставленных нам исторических данные о поведении клиентов и расторжении договоров с банком.
Также необходимо построить модель с предельно большим значением F1-меры, от 0.59. Проверим F1-меру на тестовой выборке.
Дополнительно измеряем AUC-ROC, сравнив её значение с F1-мерой.

## План проекта.

1. Загрузить и подготовить данные. Пояснить порядок действий.
2. Исследовать баланс классов, обучить модель без учёта дисбаланса. Кратко описать выводы.
3. Улучшить качество модели, учитывая дисбаланс классов. Обучить разные модели и найти лучшую. Кратко описать выводы.
4. Провести финальное тестирование на тестовой выборке и сравнить с AUC-ROC.



## Описание данных.

Признаки:

- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — количество недвижимости у клиента
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

Целевой признак:

- Exited — факт ухода клиента

## 1. Загрузка данных, необходимых библиотек и подготовка данных к анализу.

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from IPython.display import display

In [8]:
df = pd.read_csv('/datasets/Churn.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


14 столбцов, по 10000 строчек. В столбце tenure есть пропуски - необходимо их рассмотреть. 

Рассмотрим столбец tenure. 

In [9]:
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [10]:
df.query('Tenure == 0')

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
29,30,15656300,Lucciano,411,France,Male,29,0.0,59697.17,2,1,1,53483.21,0
35,36,15794171,Lombardo,475,France,Female,45,0.0,134264.04,1,1,0,27822.99,1
57,58,15647091,Endrizzi,725,Germany,Male,19,0.0,75888.20,1,0,0,45613.75,0
72,73,15812518,Palermo,657,Spain,Female,37,0.0,163607.18,1,0,1,44203.55,0
127,128,15782688,Piccio,625,Germany,Male,56,0.0,148507.24,1,1,0,46824.08,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9793,9794,15772363,Hilton,772,Germany,Female,42,0.0,101979.16,1,1,0,90928.48,0
9799,9800,15722731,Manna,653,France,Male,46,0.0,119556.10,1,1,0,78250.13,1
9843,9844,15778304,Fan,646,Germany,Male,24,0.0,92398.08,1,1,1,18897.29,0
9868,9869,15587640,Rowntree,718,France,Female,43,0.0,93143.39,1,1,0,167554.86,0


Судя по всему пропуски в столбце tenure просто означают, что у клиента нет недвижимости. Заполним нулями.

In [11]:
df['Tenure'] = df['Tenure'].fillna(0)

Для обучения модели и для анализа нам не потребуются столбцы RowNumber, CustomerId и Surname - эти признаки будут нерелевантными и собьют показатели. Отцепим их.

In [12]:
df = df.drop(['CustomerId', 'Surname', 'RowNumber'], axis=1)

Преобразуем признаки при помощи OHE. Удалим лишние столбцы.

In [13]:
df = pd.get_dummies(df, drop_first=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
CreditScore          10000 non-null int64
Age                  10000 non-null int64
Tenure               10000 non-null float64
Balance              10000 non-null float64
NumOfProducts        10000 non-null int64
HasCrCard            10000 non-null int64
IsActiveMember       10000 non-null int64
EstimatedSalary      10000 non-null float64
Exited               10000 non-null int64
Geography_Germany    10000 non-null uint8
Geography_Spain      10000 non-null uint8
Gender_Male          10000 non-null uint8
dtypes: float64(3), int64(6), uint8(3)
memory usage: 732.5 KB


In [15]:
df.head(10)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
5,645,44,8.0,113755.78,2,1,0,149756.71,1,0,1,1
6,822,50,7.0,0.00,2,1,1,10062.80,0,0,0,1
7,376,29,4.0,115046.74,4,1,0,119346.88,1,1,0,0
8,501,44,4.0,142051.07,2,0,1,74940.50,0,0,0,1
9,684,27,2.0,134603.88,1,1,1,71725.73,0,0,0,1


Данные готовы, можем переходить к анализу.

## 2. Исследуем баланс классов, обучим модель без учёта дисбаланса. 

Класс для анализа, то есть целевой признак - столбец Exited.

Классы для помощи в анализе - все остальные. Само собой, что для принятия решения моделью о том, что клиент останется или уйдет из банка, они будут иметь разный вес
Попробуем обучить модели без учета этого дисбаланса. Воспользуемся библиотекой train_test_split из sklearn.model_selection. Так как спрятанной тестовой выборки нет, то данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1. Разделим исходную таблицу на обучающую(60%) и валидационную(40%), а после разделим валидационную на конечную валидационную(20%) и тестовую(20%). 

In [16]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

Проверим разделение.

In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6000 entries, 7479 to 4578
Data columns (total 12 columns):
CreditScore          6000 non-null int64
Age                  6000 non-null int64
Tenure               6000 non-null float64
Balance              6000 non-null float64
NumOfProducts        6000 non-null int64
HasCrCard            6000 non-null int64
IsActiveMember       6000 non-null int64
EstimatedSalary      6000 non-null float64
Exited               6000 non-null int64
Geography_Germany    6000 non-null uint8
Geography_Spain      6000 non-null uint8
Gender_Male          6000 non-null uint8
dtypes: float64(3), int64(6), uint8(3)
memory usage: 486.3 KB


60%

In [18]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 8532 to 6895
Data columns (total 12 columns):
CreditScore          2000 non-null int64
Age                  2000 non-null int64
Tenure               2000 non-null float64
Balance              2000 non-null float64
NumOfProducts        2000 non-null int64
HasCrCard            2000 non-null int64
IsActiveMember       2000 non-null int64
EstimatedSalary      2000 non-null float64
Exited               2000 non-null int64
Geography_Germany    2000 non-null uint8
Geography_Spain      2000 non-null uint8
Gender_Male          2000 non-null uint8
dtypes: float64(3), int64(6), uint8(3)
memory usage: 162.1 KB


20%

In [19]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 7041 to 3366
Data columns (total 12 columns):
CreditScore          2000 non-null int64
Age                  2000 non-null int64
Tenure               2000 non-null float64
Balance              2000 non-null float64
NumOfProducts        2000 non-null int64
HasCrCard            2000 non-null int64
IsActiveMember       2000 non-null int64
EstimatedSalary      2000 non-null float64
Exited               2000 non-null int64
Geography_Germany    2000 non-null uint8
Geography_Spain      2000 non-null uint8
Gender_Male          2000 non-null uint8
dtypes: float64(3), int64(6), uint8(3)
memory usage: 162.1 KB


20%

Приступим к обучению. Для исследования качества моделей нам необходимо определить на какой информации модель учится и какие ответы ищет. Определим для нее features и target на основе обучающей выборки.

In [20]:
features = df_train.drop(['Exited'], axis=1)
target = df_train['Exited']

Также определеим сразу features и target для тестовой и валидационной выборок.

In [21]:
features_test = df_test.drop(['Exited'], axis=1)
target_test = df_test['Exited']
features_valid = df_valid.drop(['Exited'], axis=1)
target_valid = df_valid['Exited']

Начнем с модели решающего дерева. При помощи цикла переберем разную глубину и получим наиболее высокое значение f1.

In [22]:
n1 = []
ar1=[]
f1_1=[]

for n in range(2, 200):
    model_dec_tree = DecisionTreeClassifier(random_state=12345, max_depth=n)
    model_dec_tree.fit(features, target)
    predicted_valid_dec_tree = model_dec_tree.predict(features_valid)
    f1_1.append(f1_score(target_valid, predicted_valid_dec_tree))
    probabilities_valid_dec_tree = model_dec_tree.predict_proba(features_valid)
    probabilities_one_valid_dec_tree = probabilities_valid_dec_tree[:, 1]
    n1.append(n)
    ar1.append(roc_auc_score(target_valid, probabilities_one_valid_dec_tree))

    

f1_1 = pd.DataFrame(f1_1, columns = ['f1'])
n1 = pd.DataFrame(n1, columns = ['n'])
ar1 = pd.DataFrame(ar1, columns = ['auc_roc'])
mdt = f1_1.join(n1)
mdt = mdt.join(ar1)
mdt[mdt['f1']==mdt['f1'].max()]


,f1,n,auc_roc
7,0.578652,9,0.789972


Наибольшее возможное значение f1 мы получили при глубине 7 - 0.578. Auc-Roc при этом равен 0.789. Недостаточно, переходим к следующей модели.

Перейдем к модели случайного леса. Библиотека RandomForestClassifier уже импортирована. Создадим для этой модели имя model_random_tree_class, зафиксируем псевдослучайность алгоритма на показателе 12345, а для различных показателей количества деревьев напишем цикл - подберем модель с наивысшем accuracy. 

In [23]:
n2 = []
ar2=[]
f1_2=[]

for n in range(1, 200):
    model_random_tree_class = RandomForestClassifier(random_state=12345, n_estimators=n)
    model_random_tree_class.fit(features, target)
    predicted_valid_rand_tree = model_random_tree_class.predict(features_valid)
    probabilities_valid_rand_tree = model_random_tree_class.predict_proba(features_valid)
    probabilities_one_valid_rand_tree = probabilities_valid_rand_tree[:, 1]
    ar2.append(roc_auc_score(target_valid, probabilities_one_valid_rand_tree))
    f1_2.append(f1_score(target_valid, predicted_valid_rand_tree))
    n2.append(n)
 
    

    
f1_2 = pd.DataFrame(f1_2, columns = ['f1'])
n2 = pd.DataFrame(n2, columns = ['n'])
ar2 = pd.DataFrame(ar2, columns = ['auc_roc'])
mrt = f1_2.join(n2)
mrt = mrt.join(ar2)
mrt[mrt['f1']==mrt['f1'].max()]


,f1,n,auc_roc
40,0.595065,41,0.837726


Лучший показатель у модели с 41 деревом - f1 = 0.595, Auc-Roc = 0.837. Выше требуемых показателей, запомним эту модель.

Теперь попробуем логистическую регрессию.

In [24]:
model_log_reg = LogisticRegression(random_state=12345, solver='liblinear')
model_log_reg.fit(features, target)
predictions_log_reg = model_random_tree_class.predict(features_valid)
print(accuracy_score(target_valid, predictions_log_reg))

0.857


Показатель - 0.8525. Очень схоже с лесом и деревом. Оценим метрики.

In [25]:
predicted_valid_log_reg = model_log_reg.predict(features_valid)
probabilities_valid_log_reg = model_log_reg.predict_proba(features_valid)
probabilities_one_valid_log_reg = probabilities_valid_log_reg[:, 1]
auc_roc_log_reg = roc_auc_score(target_valid, probabilities_one_valid_log_reg)

print("Auc-Roc:", auc_roc_log_reg)
print("f1:", f1_score(target_valid, predicted_valid_log_reg))

Auc-Roc: 0.6727584246214894
f1: 0.08385744234800838


f1 очень плох. Требуемые модели не найдены.

Проверим дисбаланс целевого признака. Если всех клиентов в столбце Exited принять за 1, то доля клиентов оставшихся в банке будет показателем, от которого надо отталкиваться. Посчитаем это значение.


In [26]:
adeq = len(df.query('Exited == 0')) / len(df['Exited'])
adeq

0.7963

Получаем отметку в 0.7963, что свидетельствует о сильном дисбалансе целевого признака. Необходимо сбалансировать классы и уже после этого искать подходящую модель.

## Вывод

Перебрав модели и изучив их основные метрики можно придти к заключению, что дисбаланс исходных данных слишком сильно влияет на точность и полноту в худшую сторону. Но при этой данности лучше всего себя показывает модель случайного леса. Необходимо сбалансировать классы и продолжить исследование.

## 3. Улучшим качество модели, учитывая дисбаланс классов. Обучим разные модели и найдем лучшую. 

Начнем с логистической регрессии, используем удобный параметр class_weight, способный самостоятельно сбалансировать классы. Сразу посчитаем ее f1 и auc-roc.

In [27]:
model_log_reg = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model_log_reg.fit(features, target)
predictions_log_reg = model_log_reg.predict(features_valid)
predicted_valid_log_reg = model_log_reg.predict(features_valid)
print("f1:", f1_score(target_valid, predicted_valid_log_reg))
probabilities_valid_log_reg = model_log_reg.predict_proba(features_valid)
probabilities_one_valid_log_reg = probabilities_valid_log_reg[:, 1]
auc_roc_log_reg = roc_auc_score(target_valid, probabilities_one_valid_log_reg)
print("Auc-Roc:", auc_roc_log_reg)

f1: 0.48833333333333334
Auc-Roc: 0.7542765804293519


Достаточно далеко от требуемых значений. Перейдем к следующей модели.

Изучим случайный лес с тем же параметром.

In [28]:
n3 = []
ar3=[]
f1_3=[]

for n in range(1, 200):
    model_random_tree_class = RandomForestClassifier(random_state=12345, n_estimators=n, class_weight = 'balanced')
    model_random_tree_class.fit(features, target)
    predicted_valid_rand_tree = model_random_tree_class.predict(features_valid)
    probabilities_valid_rand_tree = model_random_tree_class.predict_proba(features_valid)
    probabilities_one_valid_rand_tree = probabilities_valid_rand_tree[:, 1]
    ar3.append(roc_auc_score(target_valid, probabilities_one_valid_rand_tree))
    f1_3.append(f1_score(target_valid, predicted_valid_rand_tree))
    n3.append(n)
 
    

    
f1_3 = pd.DataFrame(f1_3, columns = ['f1'])
n3 = pd.DataFrame(n3, columns = ['n'])
ar3 = pd.DataFrame(ar3, columns = ['auc_roc'])
mrtb = f1_3.join(n3)
mrtb = mrtb.join(ar3)
mrtb[mrtb['f1']==mrtb['f1'].max()]


,f1,n,auc_roc
46,0.571429,47,0.835791


Значительно лучше, чем у логистической регрессии - f1 = 0.571 и Auc-Roc = 0.835 при 47 деревьях. Но все равно недостаточно.

Попробуем обучить модель решающего дерева.

In [29]:
n4 = []
ar4=[]
f1_4=[]

for n in range(2, 200):
    model_dec_tree = DecisionTreeClassifier(random_state=12345, max_depth=n, class_weight = 'balanced')
    model_dec_tree.fit(features, target)
    predicted_valid_dec_tree = model_dec_tree.predict(features_valid)
    f1_4.append(f1_score(target_valid, predicted_valid_dec_tree))
    probabilities_valid_dec_tree = model_dec_tree.predict_proba(features_valid)
    probabilities_one_valid_dec_tree = probabilities_valid_dec_tree[:, 1]
    n4.append(n)
    ar4.append(roc_auc_score(target_valid, probabilities_one_valid_dec_tree))

    

f1_4 = pd.DataFrame(f1_4, columns = ['f1'])
n4 = pd.DataFrame(n4, columns = ['n'])
ar4 = pd.DataFrame(ar4, columns = ['auc_roc'])
mdtb = f1_4.join(n4)
mdtb = mdtb.join(ar4)
mdtb[mdtb['f1']==mdtb['f1'].max()]   

,f1,n,auc_roc
3,0.596379,5,0.831024


Нам необходимо выбрать наиболее перспективную модель, учитывая дисбаланс классов, чтобы взять ее на тестовый прогон. Определяющей метрикой является f1 и по ней предпочтительнее выглядит модель решающего дерева с глубиной 5, f1 равным 0.596 и Auc-Roc равным 0.831. Очевидный фаворит. Забираем.

Теперь попробуем достичь максимально показателя f1 путем увеличения выборки. Начнем с логистической регрессии. Сначала напишем функцию для увеличения выборки, а после попробуем подобрать модель.

In [31]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features, target, 4)

In [32]:
model_log_reg = LogisticRegression(random_state=12345, solver='liblinear')
model_log_reg.fit(features_upsampled, target_upsampled)
predictions_log_reg = model_log_reg.predict(features_valid)
predicted_valid_log_reg = model_log_reg.predict(features_valid)
print("f1:", f1_score(target_valid, predicted_valid_log_reg))
probabilities_valid_log_reg = model_log_reg.predict_proba(features_valid)
probabilities_one_valid_log_reg = probabilities_valid_log_reg[:, 1]
auc_roc_log_reg = roc_auc_score(target_valid, probabilities_one_valid_log_reg)
print("Auc-Roc:", auc_roc_log_reg)

f1: 0.45286059629331177
Auc-Roc: 0.719318408652363


f1 = 0.460 - неудача. Идем в случайный лес.

In [33]:
n5 = []
ar5=[]
f1_5=[]

for n in range(1, 200):
    model_random_tree_class = RandomForestClassifier(random_state=12345, n_estimators=n)
    model_random_tree_class.fit(features_upsampled, target_upsampled)
    predicted_valid_rand_tree = model_random_tree_class.predict(features_valid)
    probabilities_valid_rand_tree = model_random_tree_class.predict_proba(features_valid)
    probabilities_one_valid_rand_tree = probabilities_valid_rand_tree[:, 1]
    ar5.append(roc_auc_score(target_valid, probabilities_one_valid_rand_tree))
    f1_5.append(f1_score(target_valid, predicted_valid_rand_tree))
    n5.append(n)
 
    

    
f1_5 = pd.DataFrame(f1_5, columns = ['f1'])
n5 = pd.DataFrame(n5, columns = ['n'])
ar5 = pd.DataFrame(ar5, columns = ['auc_roc'])
mrtu = f1_5.join(n5)
mrtu = mrtu.join(ar5)
mrtu[mrtu['f1']==mrtu['f1'].max()]

,f1,n,auc_roc
178,0.611765,179,0.838189


Есть улучшение. При параметре количества деревьев 179 мы получили пиковый показатель f1 в 0.611, Auc-Roc равен 0.838. Пока лучший результат. Перейдем к решающему дереву.

In [34]:
n6 = []
ar6=[]
f1_6=[]

for n in range(2, 10):
    model_dec_tree_up = DecisionTreeClassifier(random_state=12345, max_depth=n)
    model_dec_tree_up.fit(features_upsampled, target_upsampled)
    predicted_valid_dec_tree = model_dec_tree_up.predict(features_valid)
    f1_6.append(f1_score(target_valid, predicted_valid_dec_tree))
    probabilities_valid_dec_tree = model_dec_tree_up.predict_proba(features_valid)
    probabilities_one_valid_dec_tree = probabilities_valid_dec_tree[:, 1]
    n6.append(n)
    ar6.append(roc_auc_score(target_valid, probabilities_one_valid_dec_tree))

    

f1_6 = pd.DataFrame(f1_6, columns = ['f1'])
n6 = pd.DataFrame(n6, columns = ['n'])
ar6 = pd.DataFrame(ar6, columns = ['auc_roc'])
mdtu = f1_6.join(n6)
mdtu = mdtu.join(ar6)
mdtu[mdtu['f1']==mdtu['f1'].max()]


,f1,n,auc_roc
3,0.596379,5,0.831024


Здесь не получилось того же эффекта. Перейдем к уменьшению выборки. Напишем функцию, уменьшим выборку и запустим модели.

In [36]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features, target, 0.25)

И вновь, начнем с логистической регрессии.

In [37]:
model_log_reg = LogisticRegression(random_state=12345, solver='liblinear')
model_log_reg.fit(features_downsampled, target_downsampled)
predictions_log_reg = model_log_reg.predict(features_valid)
predicted_valid_log_reg = model_log_reg.predict(features_valid)
print("f1:", f1_score(target_valid, predicted_valid_log_reg))
probabilities_valid_log_reg = model_log_reg.predict_proba(features_valid)
probabilities_one_valid_log_reg = probabilities_valid_log_reg[:, 1]
auc_roc_log_reg = roc_auc_score(target_valid, probabilities_one_valid_log_reg)
print("Auc-Roc:", auc_roc_log_reg)

f1: 0.45337620578778143
Auc-Roc: 0.7262822180148683


Также как и у upsampling. Идем в лес.

In [38]:
n7 = []
ar7=[]
f1_7=[]

for n in range(1, 200):
    model_random_tree_class = RandomForestClassifier(random_state=12345, n_estimators=n)
    model_random_tree_class.fit(features_downsampled, target_downsampled)
    predicted_valid_rand_tree = model_random_tree_class.predict(features_valid)
    probabilities_valid_rand_tree = model_random_tree_class.predict_proba(features_valid)
    probabilities_one_valid_rand_tree = probabilities_valid_rand_tree[:, 1]
    ar7.append(roc_auc_score(target_valid, probabilities_one_valid_rand_tree))
    f1_7.append(f1_score(target_valid, predicted_valid_rand_tree))
    n7.append(n)
 
    

    
f1_7 = pd.DataFrame(f1_7, columns = ['f1'])
n7 = pd.DataFrame(n7, columns = ['n'])
ar7 = pd.DataFrame(ar7, columns = ['auc_roc'])
mrtd = f1_7.join(n7)
mrtd = mrtd.join(ar7)
mrtd[mrtd['f1']==mrtd['f1'].max()]

,f1,n,auc_roc
49,0.596759,50,0.843749


Показатели хуже, чем при увеличении выборки. Может решающее дерево лучше отреагирует на этот метод?

In [39]:
n8 = []
ar8=[]
f1_8=[]

for n in range(2, 200):
    model_dec_tree = DecisionTreeClassifier(random_state=12345, max_depth=n)
    model_dec_tree.fit(features_downsampled, target_downsampled)
    predicted_valid_dec_tree = model_dec_tree.predict(features_valid)
    f1_8.append(f1_score(target_valid, predicted_valid_dec_tree))
    probabilities_valid_dec_tree = model_dec_tree.predict_proba(features_valid)
    probabilities_one_valid_dec_tree = probabilities_valid_dec_tree[:, 1]
    n8.append(n)
    ar8.append(roc_auc_score(target_valid, probabilities_one_valid_dec_tree))

    

f1_8 = pd.DataFrame(f1_8, columns = ['f1'])
n8 = pd.DataFrame(n8, columns = ['n'])
ar8 = pd.DataFrame(ar8, columns = ['auc_roc'])
mdtd = f1_8.join(n8)
mdtd = mdtd.join(ar8)
mdtd[mdtd['f1']==mdtd['f1'].max()]   

,f1,n,auc_roc
3,0.593117,5,0.822914


К сожалению решающее дерево не поддалось нашим уговорам.

## Вывод.

В попытках нивелировать дисбаланс классов мы сначала оценили основные метрики на моделях логистической регрессии, решающего дерева и случайного леса при значении "balanced" параметра class_weight. Далее мы приступили к увеличению и уменьшению выборок на всех моделях. Оптимальным для нас оказалось увеличение выборки на модели случайного леса с 178 деревьями - достигнуто значение f1 в 0.611, что больше требуемых 0.59, а Auc-Roc равен 0.838.

## 4. Проведем финальное тестирование на тестовой выборке и сравним с AUC-ROC.

Приступим к финальному тестированию выбранной нами модели. Проведем его на тестовой выборке, посчитаем основные метрики и сравним f1 с Auc-Roc.

In [40]:
model_up = RandomForestClassifier(random_state=12345, n_estimators=178)
model_up.fit(features_upsampled, target_upsampled)    
predictions_rand_tree = model_up.predict(features_test)
predicted_test_rand_tree = model_up.predict(features_test)
print("f1:", f1_score(target_test, predicted_test_rand_tree))
probabilities_test_rand_tree = model_up.predict_proba(features_test)
probabilities_one_test_rand_tree = probabilities_test_rand_tree[:, 1]
auc_roc_rand_tree = roc_auc_score(target_test, probabilities_one_test_rand_tree)
print("Auc-Roc:", auc_roc_rand_tree)

f1: 0.5873015873015872
Auc-Roc: 0.8527285701222208


Значение f1 на тестовой выборке снизилось на 0.024, до 0.587 и оно существенно меньше параметра Auc-Roc - 0.852. Нет требуемого показателя. Попробуем пойти от отбратного и напишем формулу для поиска удволетворяющей нас комбинации.

In [41]:
n9= []
ar9=[]
f1_9=[]

for n in range(1, 200):
    model_random_tree_class_final = RandomForestClassifier(random_state=12345, n_estimators=n)
    model_random_tree_class_final.fit(features_upsampled, target_upsampled)
    predicted_test_rand_tree = model_random_tree_class_final.predict(features_test)
    probabilities_test_rand_tree = model_random_tree_class_final.predict_proba(features_test)
    probabilities_one_test_rand_tree = probabilities_test_rand_tree[:, 1]
    ar9.append(roc_auc_score(target_test, probabilities_one_test_rand_tree))
    f1_9.append(f1_score(target_test, predicted_test_rand_tree))
    n9.append(n)
 
    

    
f1_9 = pd.DataFrame(f1_9, columns = ['f1_test'])
n9 = pd.DataFrame(n9, columns = ['n_test'])
ar9 = pd.DataFrame(ar9, columns = ['auc_roc_test'])
mrtuf = f1_9.join(n9)
mrtuf = mrtuf.join(ar9)
final = mrtu.join(mrtuf)
final.query('f1 > 0.59 and f1_test > 0.59')

,f1,n,auc_roc,f1_test,n_test,auc_roc_test
110,0.603974,111,0.837504,0.591327,111,0.849963
111,0.600266,112,0.837668,0.590728,112,0.849721
112,0.603175,113,0.837768,0.593176,113,0.850097
114,0.603175,115,0.837744,0.591029,115,0.850344
116,0.604222,117,0.837997,0.594737,117,0.850460
117,0.599469,118,0.838093,0.592593,118,0.850508
118,0.604222,119,0.838083,0.591029,119,0.850482
119,0.599469,120,0.838232,0.592593,120,0.850530
120,0.603426,121,0.838109,0.591810,121,0.850484
122,0.601840,123,0.838151,0.592885,123,0.850538


Как оказалось, таких комбинаций предостаточно, просто нам не повезло их найти ранее. Лучшая из них - модель случайного леса со 133 деревьями.

## Вывод по проекту.

Заказчиком выступил Бета-Банк с задачей построить модель, помогающую определить вероятность ухода клиента из банка в ближайшее время на основе имеющихся данных за некоторое прошедшее время. 
Подготовив данные, убрав из них нерелевантную информацию, заполнив пропуски и преобразовав признаки, мы приступили к анализу. 
Целевым признаком выступает столбец Exited, содержащий информацию о том ушел клиент или остался. Сначала мы пытались построить модели, которые не будут принимать во внимание дисбаланс между классами, но этот подход не показал удовлетворительных значений нужных нам метрик.
Далее путем нивелирования баланса между классами в разных моделях мы пришли к модели случайного леса, которая оказалась самой точной среди написанных с весьма высокими показателями качества по метрикам f1 и Auc-Roc. 

Объем данных для анализа оказался недостаточным для изучения, чтобы получить 100% вероятность предсказания уйдет тот или иной клиент или нет. Поэтому опираться при принятия решения на созданную нами модель можно, но только как на косвенный признак. 

На наш взгляд, для того чтобы избежать утечки клиентов, Бета-Банку необходимо провести анализ удовлетворенности  среди клиентов, имеющих счета в других банках, или клиентов имеющих положительный баланс на счету более 1000$, но с минимальным количеством операций. Возможно этот анализ даст более ясные причины вероятного ухода, а соответственно и поможет сделать более точный прогноз.